In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [46]:
header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
res = requests.get("https://www.ura.gov.sg/realEstateIIWeb/resources/misc/list_of_postal_districts.htm",headers=header)
html_content = res.content

soup = BeautifulSoup(html_content, 'html.parser')

table = soup.find("table")
rows = table.find_all("tr")

# Extract the data and store in a list
data = []
for row in rows:
    cells = row.find_all("td")
    if cells:
        postal_district = cells[0].text
        postal_sector = cells[1].text
        location = cells[2].text
        data.append([postal_district.strip().replace("\r\n", ""), postal_sector.strip().replace("\r\n", ""), location.strip().replace("\r\n", "")])

# Create a pandas dataframe from the list
import pandas as pd
df = pd.DataFrame(data, columns=["Postal District", "Postal Sector", "Location"])
df = df.drop(df.index[0])
df


,Postal District,Postal Sector,Location
1,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
2,02,"07, 08","Anson, Tanjong Pagar"
3,03,"14,15, 16","Queenstown, Tiong Bahru"
4,04,"09, 10","Telok Blangah, Harbourfront"
5,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clement..."
6,06,17,"High Street, Beach Road (part)"
7,07,"18, 19","Middle Road, Golden Mile"
8,08,"20, 21",Little India
9,09,"22, 23","Orchard, Cairnhill, River Valley"
10,10,"24, 25, 26, 27","Ardmore, Bukit Timah, Holland Road, Tanglin"


In [47]:
# algorithm to split the values into rows
newDf = pd.DataFrame([],columns=["Postal Dist","Postal Sect"],index=[0])
for index,row in df.iterrows():
    tempDist = row["Postal District"]
    tempSect = row["Postal Sector"].split(",")
    for i in tempSect:
        tempObj = pd.DataFrame({"Postal Dist":tempDist,
                                "Postal Sect":int(i)},index=[0])
        newDf = pd.concat([newDf,tempObj],ignore_index=True)
newDf = newDf[newDf.index!=0]
newDf

,Postal Dist,Postal Sect
1,01,1
2,01,2
3,01,3
4,01,4
5,01,5
...,...,...
77,26,78
78,27,75
79,27,76
80,28,79


In [48]:
newDf[newDf["Postal Sect"].duplicated()] # no duplicates here
newDf = newDf.sort_values("Postal Sect")
newDf = newDf.reset_index(drop=True)

In [49]:

newDf.to_csv("./wrangled/postalToDist.csv")